In [1]:
#Gerekli kütüphaneler ekliyoruz
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime as dt

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 500)

In [3]:
# veri seti okuyoruz
data_original = pd.read_excel("D://verisetleri//online_retail_2.xlsx")

In [187]:
# orjinal veri setinin bozulmasını engellemk için kopyasını çıkartıyoruz.
data = data_original.copy()

In [189]:
# ilk beş satırını gösterir
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [191]:
# son 5 satırı gösterir
data.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525460,538171,21931,JUMBO STORAGE BAG SUKI,2,2010-12-09 20:01:00,1.95,17530.0,United Kingdom


In [193]:
# veri setinnin değişkenleri hakkında bilgi verir
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [194]:
# veri setinin sayısal değişkenlerinin istatistiki bilgilerini verir
data.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,525461.0,10.337667,-9600.0,1.0,3.0,10.0,19152.0,107.42411
InvoiceDate,525461,2010-06-28 11:37:36.845017856,2009-12-01 07:45:00,2010-03-21 12:20:00,2010-07-06 09:51:00,2010-10-15 12:45:00,2010-12-09 20:01:00,NaN
Price,525461.0,4.688834,-53594.36,1.25,2.1,4.21,25111.09,146.126914
Customer ID,417534.0,15360.645478,12346.0,13983.0,15311.0,16799.0,18287.0,1680.811316


In [195]:
# değişkenlerde eksik veri varsa gösterir
data.isnull().values.any()

True

In [196]:
# hangi değişkenlerde ne kadar eksik veri olduğunu gösterir
data.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [199]:
# veri setindeki değişken isimlerini gösterir
data.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [200]:
# değişken isimlerini küçük harfe çevirir
data.columns = data.columns.str.lower()

In [202]:
# değişken isimleri arasıdna boşluk varsa " _ " ile değiştirir.
data.columns = data.columns.str.replace(" ", "_").str.replace("[^a-z0-9_]", "", regex=True)

In [207]:
# veri setinde kaç değişken(8) ve kaç satır(525461) olduğunu gösterir
data.shape

(525461, 8)

In [209]:
#veri setindeki eşsiz veri değerlerinin sayısını bulalım.
print("Description :",data["description"].nunique())
print("Country :",data["country"].nunique())
print("StockCode :",data["stockcode"].nunique())
print("Invoice :",data["invoice"].nunique())
print("Customer ID :",data["customer_id"].nunique())

Description : 4681
Country : 40
StockCode : 4632
Invoice : 28816
Customer ID : 4383


In [211]:
#Veri setindeki en çok olan ilk 10 ürün?
data["description"].value_counts().head(10)

description
WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
60 TEATIME FAIRY CAKE CASES           1400
HOME BUILDING BLOCK WORD              1386
JUMBO BAG RED RETROSPOT               1310
LUNCH BAG RED SPOTTY                  1274
REX CASH+CARRY JUMBO SHOPPER          1232
Name: count, dtype: int64

In [213]:
data.groupby("description").agg({"quantity":"sum"}).sort_values("quantity",ascending = False).head(10)

,quantity
description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925
60 TEATIME FAIRY CAKE CASES,36326
PACK OF 60 PINK PAISLEY CAKE CASES,31822
JUMBO BAG RED RETROSPOT,30727
SMALL POPCORN HOLDER,29500


In [215]:
# bir ürün için toplamda ne kadar ödeme yapıldığını görmek için yeni bir değişken yarattık
data["totalprice"] =data["quantity"] * data["price"]

In [217]:
data.head()

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,totalprice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [219]:
# veri setinde eksik verileri kaldıralım
data_full = data.dropna()

In [221]:
data_full.isnull().sum()

invoice        0
stockcode      0
description    0
quantity       0
invoicedate    0
price          0
customer_id    0
country        0
totalprice     0
dtype: int64

In [223]:
data_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      417534 non-null  object        
 1   stockcode    417534 non-null  object        
 2   description  417534 non-null  object        
 3   quantity     417534 non-null  int64         
 4   invoicedate  417534 non-null  datetime64[ns]
 5   price        417534 non-null  float64       
 6   customer_id  417534 non-null  float64       
 7   country      417534 non-null  object        
 8   totalprice   417534 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 31.9+ MB


In [225]:
# bazı faturalar iade faturası ve fatura numaralarının başında C harfi bulunmakta. Bu faturaların sayısını bulalım.
data_full[data_full["invoice"].astype(str).str.contains(r"[A-Za-z]")]["invoice"].shape[0]

9839

In [226]:
# İçinde harf olan fatura numaralarını filtrele
harf_iceren_faturalar = data_full[data_full["invoice"].astype(str).str.contains(r"[A-Za-z]")]["invoice"]

# Tüm harfleri birleştir ve eşsiz harfleri bul
farklı_harfler = set("".join(harf_iceren_faturalar.astype(str)).upper()) & set("ABCDEFGHIJKLMNOPQRSTUVWXYZ")

# Sonucu görüntüle
print(f"Fatura numaralarında geçen farklı harfler: {farklı_harfler}")
print(f"Toplam farklı harf sayısı: {len(farklı_harfler)}")

Fatura numaralarında geçen farklı harfler: {'C'}
Toplam farklı harf sayısı: 1


In [229]:
# iade faturalarının başından C harfini çıkartacağız ve fatura durumu adında yeni bir değişken yaratıp fatura durmunu buraya kayıt edeceğiz.

In [231]:
# invoicestatus değişkenini oluştur
data_full["invoicestatus"] = data_full["invoice"].astype(str).str.startswith("C").astype(int)

In [233]:
# rastgele 10 tane veriye göz atalım
data_full.sample(10)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,totalprice,invoicestatus
311704,519731,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2,2010-08-19 19:50:00,2.95,13542.0,United Kingdom,5.90,0
249150,513492,84755,COLOUR GLASS T-LIGHT HOLDER HANGING,16,2010-06-25 09:36:00,0.65,16883.0,United Kingdom,10.40,0
344952,522987,22637,PIGGY BANK RETROSPOT,6,2010-09-19 14:51:00,2.55,17625.0,United Kingdom,15.30,0
101224,498935,22114,HOT WATER BOTTLE TEA AND SYMPATHY,8,2010-02-24 10:42:00,3.95,14911.0,EIRE,31.60,0
260090,514498,84596B,SMALL DOLLY MIX DESIGN ORANGE BOWL,1,2010-07-04 10:45:00,1.25,16782.0,United Kingdom,1.25,0
242313,512827,22087,PAPER BUNTING WHITE LACE,6,2010-06-18 11:34:00,2.95,15981.0,United Kingdom,17.70,0
122092,501049,22432,WATERING CAN PINK BUNNY,96,2010-03-12 10:41:00,1.65,14646.0,Netherlands,158.40,0
108170,499771,21175,GIN + TONIC DIET METAL SIGN,1,2010-03-02 13:28:00,2.10,18283.0,United Kingdom,2.10,0
3186,489665,85071A,BLUE CHARLIE+LOLA PERSONAL DOORSIGN,6,2009-12-01 18:07:00,2.95,14108.0,United Kingdom,17.70,0
399211,527512,22734,SET OF 6 RIBBONS VINTAGE CHRISTMAS,5,2010-10-18 12:54:00,2.55,14130.0,United Kingdom,12.75,0


In [241]:
# "C" harfini sil ve sayısal tipe çevir
data_full["invoice"] = data_full["invoice"].astype(str).str.replace("C", "", regex=True).astype(int)

In [251]:
# rastgele 10 tane veriye göz atalım
data_full.sample(10)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,totalprice,invoicestatus
225882,511266,21611,SET OF 12 LILY BOTANICAL T-LIGHTS,1,2010-06-07 12:51:00,2.95,14534.0,United Kingdom,2.95,0
261480,514567,22355,CHARLOTTE BAG SUKI DESIGN,10,2010-07-05 11:56:00,0.85,15627.0,United Kingdom,8.50,0
416222,529156,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,7,2010-10-26 16:47:00,2.55,16549.0,United Kingdom,17.85,0
20860,491055,85123A,WHITE HANGING HEART T-LIGHT HOLDER,12,2009-12-09 10:48:00,2.95,14732.0,United Kingdom,35.40,0
474277,534054,22453,MEASURING TAPE BABUSHKA BLUE,3,2010-11-21 10:38:00,2.95,16746.0,United Kingdom,8.85,0
200944,508695,79323P,PINK CHERRY LIGHTS,-7,2010-05-17 17:01:00,6.75,13327.0,United Kingdom,-47.25,1
230433,511655,22271,FELTCRAFT DOLL ROSIE,1,2010-06-09 14:09:00,2.95,14104.0,United Kingdom,2.95,0
272847,515907,84279P,CHERRY BLOSSOM DECORATIVE FLASK,1,2010-07-15 13:13:00,3.75,13564.0,United Kingdom,3.75,0
403002,527963,85123A,WHITE HANGING HEART T-LIGHT HOLDER,3,2010-10-20 09:41:00,2.95,17894.0,United Kingdom,8.85,0
150125,503710,21713,CITRONELLA CANDLE FLOWERPOT,8,2010-04-06 14:18:00,2.10,16266.0,United Kingdom,16.80,0


In [253]:
# verileri gözden geçirelim
data_full.describe().T

,count,mean,min,25%,50%,75%,max,std
invoice,417534.0,514714.691532,489434.0,502724.0,515181.0,527068.0,538171.0,14102.79281
quantity,417534.0,12.758815,-9360.0,2.0,4.0,12.0,19152.0,101.220424
invoicedate,417534,2010-07-01 01:12:19.787609856,2009-12-01 07:45:00,2010-03-26 11:26:00,2010-07-08 19:12:00,2010-10-14 14:08:00,2010-12-09 20:01:00,NaN
price,417534.0,3.887547,0.0,1.25,1.95,3.75,25111.09,71.131797
customer_id,417534.0,15360.645478,12346.0,13983.0,15311.0,16799.0,18287.0,1680.811316
totalprice,417534.0,19.994081,-25111.09,4.25,11.25,19.35,15818.4,99.915863
invoicestatus,417534.0,0.023565,0.0,0.0,0.0,0.0,1.0,0.151688


In [255]:
# Hangi ülkeden ne kadar sipariş olduğuna bakalım

In [257]:
data_full["country"].value_counts()

country
United Kingdom          379423
EIRE                      8710
Germany                   8129
France                    5710
Netherlands               2769
Spain                     1278
Switzerland               1187
Belgium                   1054
Portugal                  1024
Channel Islands            906
Sweden                     883
Italy                      731
Australia                  654
Cyprus                     554
Austria                    537
Greece                     517
Denmark                    428
Norway                     369
Finland                    354
United Arab Emirates       318
Unspecified                280
USA                        244
Japan                      224
Poland                     194
Malta                      172
Lithuania                  154
Singapore                  117
Canada                      77
Thailand                    76
Israel                      74
Iceland                     71
RSA                         65


In [259]:
# Hangi ülke ne kadar para kazandırdı?
data_full.groupby("country").agg({"totalprice":"sum"}).sort_values("totalprice",ascending = False).head(10)

,totalprice
country,
United Kingdom,7038549.633
EIRE,328216.410
Netherlands,263863.410
Germany,196290.351
France,129773.830
Sweden,50859.510
Denmark,46972.950
Switzerland,43343.410
Spain,37084.900


In [494]:
# Geçerli faturaları seç
valid_data = data_full[data_full["invoicestatus"] == 0]

In [261]:
# ------------------------- GENEL OLARAK VERİ SETİNE BAKIŞIMIZI TAMAMLADIK -----------------
# RFM Analizine geçebiliriz

In [263]:
# RECENCY

In [570]:
valid_data.head()

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,totalprice,invoicestatus
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,0
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,0
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,0


In [496]:
# ilk fatura tarihini alalım
valid_data["invoicedate"].min()

Timestamp('2009-12-01 07:45:00')

In [498]:
# son fatura tarihine bakalım
valid_data["invoicedate"].max()

Timestamp('2010-12-09 20:01:00')

In [445]:
# en son fatura tarihinden bir gün sonrayı bugün olarak alıyoruz.
today_date = dt.datetime(2011,12,10)

In [447]:
today_date

datetime.datetime(2011, 12, 10, 0, 0)

In [572]:
# En güncel fatura tarihini al
recency_data = (today_date - valid_data.groupby("customer_id").agg({"invoicedate":"max"}))

In [ ]:
recency_data.rename(columns={"invoicedate": "Recency"}, inplace=True)

In [574]:
recency_data.head()

,invoicedate
customer_id,
12346,529 days 10:07:00
12347,367 days 09:03:00
12348,438 days 09:01:00
12349,407 days 15:37:00
12351,375 days 08:37:00


In [576]:
#InvoiceDate yerine Recency olarak değişken ismini değiştirelim.
recency_data.rename(columns = {"invoicedate":"Recency"}, inplace = True)

In [578]:
recency_data = recency_data["Recency"].apply(lambda x: x.days)
recency_data.head()

customer_id
12346    529
12347    367
12348    438
12349    407
12351    375
Name: Recency, dtype: int64

In [508]:
# FREQUENCY

In [459]:
# Her bir müşterinin toplamda ne kadar alışveriş yaptığı bilgisini istiyoruz.

In [510]:
#Her bir müşterinin kaç tane faturası olduğunu gördük
valid_data.groupby(["customer_id","invoice"]).agg({"invoice":"nunique"}).head(20)

invoice
customer_id invoice         
12346       491725         1
            491742         1
            491744         1
            492718         1
            492722         1
            493410         1
            493412         1
            494450         1
            495295         1
            499763         1
            513774         1
12347       529924         1
            537626         1
12348       524140         1
12349       506394         1
            508728         1
            529344         1
12351       536012         1
12352       532430         1
            535838         1

In [512]:
# rastgele bir fatura numarasında kaç adet ürün satın alındığına bakabiliriz.
valid_data[valid_data['customer_id'] == 12346].invoice.unique()

array([491725, 491742, 491744, 492718, 492722, 493410, 493412, 494450,
       495295, 499763, 513774])

In [514]:
# frekans bilgilerini bulalım
freq_data = valid_data.groupby("customer_id").agg({"invoicedate":"nunique"})

In [516]:
freq_data.head()

,invoicedate
customer_id,
12346,11
12347,2
12348,1
12349,3
12351,1


In [518]:
#Değişken isminin Frequency olarak değiştiriyoruz
freq_data.rename(columns={"invoicedate": "Frequency"}, inplace=True)

In [520]:
freq_data.head()

,Frequency
customer_id,
12346,11
12347,2
12348,1
12349,3
12351,1


In [522]:
# MONETARY

In [524]:
# Her bir müşterinin toplam kaç para harcadığını bulalım

In [526]:
# Müşteri bazında toplam harcamayı hesapla
monetary_data = valid_data.groupby("customer_id").agg({"totalprice": "sum"})

# Kolon adını "Monetary" olarak değiştir
monetary_data.rename(columns={"totalprice": "Monetary"}, inplace=True)

In [528]:
monetary_data.head()

,Monetary
customer_id,
12346,372.86
12347,1323.32
12348,222.16
12349,2671.14
12351,300.93


In [530]:
# ELDE EDİLEN RFM BİLGİLERİNİ BİRLEŞTİRİP İNCELEME

In [590]:
# şimdi elimizde bütün değerler var.
print(recency_data.shape,freq_data.shape,monetary_data.shape)

(4314,) (4314, 1) (4314, 1)


In [588]:
# R,F ve M ile oluşturulmuş veri setlerinin birleştirilmesi
data_rfm = pd.concat([recency_data, freq_data, monetary_data], axis=1)
data_rfm.sample(10)

,Recency,Frequency,Monetary
customer_id,,,
16060,400,3,541.95
14461,382,10,4610.44
13439,388,2,702.90
14672,515,2,347.59
15030,380,6,1461.63
15550,378,8,1643.17
15129,385,12,3412.78
14989,661,1,342.40
18107,442,1,167.13


In [592]:
#Recency, Frequency ve Monetary için skor değişkenlerinin oluşturulması (qcut--> kesmeyi sağlar cut etmek isted
#Recency'de büyük olan küçüktür. Frequency ve Monetary 'de büyük olan büyüktür.
data_rfm["RecencyScore"] = pd.qcut(data_rfm["Recency"], 5, labels = [5, 4, 3, 2, 1])
data_rfm["FrequencyScore"]= pd.qcut(data_rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])
data_rfm["MonetaryScore"] = pd.qcut(data_rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])

In [594]:
data_rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
customer_id,,,,,,
12346,529,11,372.86,2,5,2
12347,367,2,1323.32,5,2,4
12348,438,1,222.16,2,1,1
12349,407,3,2671.14,3,3,5
12351,375,1,300.93,5,1,2


In [598]:
# RFM skorları kategorik değere dönüştürülüp df'e eklendi
data_rfm["RFM_SCORE"] = (data_rfm['RecencyScore'].astype(str) +
data_rfm['FrequencyScore'].astype(str) +
data_rfm['MonetaryScore'].astype(str))

In [600]:
data_rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
customer_id,,,,,,,
12346,529,11,372.86,2,5,2,252
12347,367,2,1323.32,5,2,4,524
12348,438,1,222.16,2,1,1,211
12349,407,3,2671.14,3,3,5,335
12351,375,1,300.93,5,1,2,512


In [602]:
# RFM içinde koşul çalıştırma ve çıkan gözlemlerin gösterilmesi
data_rfm[data_rfm["RFM_SCORE"]=="555"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
customer_id,,,,,,,
12415,375,7,19543.84,5,5,5,555
12431,373,13,4370.52,5,5,5,555
12471,374,48,20139.74,5,5,5,555
12472,369,13,11308.48,5,5,5,555
12474,378,13,5048.66,5,5,5,555


In [604]:
# RFM içinde koşul çalıştırma ve çıkan gözlemlerin gösterilmesi
data_rfm[data_rfm["RFM_SCORE"]=="111"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
customer_id,,,,,,,
12362,738,1,130.00,1,1,1,111
12392,591,1,234.75,1,1,1,111
12404,682,1,63.24,1,1,1,111
12416,656,1,202.56,1,1,1,111
12466,682,1,56.73,1,1,1,111


In [606]:
# RFM içinde koşul çalıştırma ve çıkan gözlemlerin gösterilmesi
data_rfm[data_rfm["RFM_SCORE"]=="333"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
customer_id,,,,,,,
12909,424,3,924.52,3,3,3,333
12987,420,3,753.51,3,3,3,333
13104,415,3,610.62,3,3,3,333
13476,406,3,823.00,3,3,3,333
13514,428,3,663.70,3,3,3,333


In [608]:
# RFM içinde koşul çalıştırma ve çıkan gözlemlerin gösterilmesi
data_rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4314.0,455.269819,96.944304,365.0,382.00,417.00,500.0000,738.00
Frequency,4314.0,4.445758,8.112355,1.0,1.00,2.00,5.0000,204.00
Monetary,4314.0,2047.288659,8912.523243,0.0,307.95,705.55,1722.8025,349164.35


In [618]:
# Regular Expressions (Düzenli İfadeler) kullanılarak RFM haritası çıkarıldı
segmentation_map = {
r'[1-2][1-2]': 'Uzun Süredir Pasif',
r'[1-2][3-4]': 'Riskli',
r'[1-2]5': 'Kaybedilmesin',
r'3[1-2]': 'Uykuda',
r'33': 'Dikkat edilsin',
r'[3-4][4-5]': 'Sadık Müşteri',
r'41': 'Umut Vaat eden',
r'51': 'Yeni Müşteri',
r'[4-5][2-3]': 'Potansiyel Sadık Müşteri',
r'5[4-5]': 'Gözde Müşteri'
}

In [620]:
#RecencyScore ve FrequencyScore değerlerinin birleştirilmesiyle Segment değişkeninin veri setine eklenmesi
data_rfm['Segment'] = data_rfm['RecencyScore'].astype(str) + data_rfm['FrequencyScore'].astype(str)

In [622]:
data_rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
customer_id,,,,,,,,
12346,529,11,372.86,2,5,2,252,25
12347,367,2,1323.32,5,2,4,524,52
12348,438,1,222.16,2,1,1,211,21
12349,407,3,2671.14,3,3,5,335,33
12351,375,1,300.93,5,1,2,512,51


In [624]:
data_rfm['Segment'] = data_rfm['Segment'].replace(segmentation_map, regex=True)
data_rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
customer_id,,,,,,,,
12346,529,11,372.86,2,5,2,252,Kaybedilmesin
12347,367,2,1323.32,5,2,4,524,Potansiyel Sadık Müşteri
12348,438,1,222.16,2,1,1,211,Uzun Süredir Pasif
12349,407,3,2671.14,3,3,5,335,Dikkat edilsin
12351,375,1,300.93,5,1,2,512,Yeni Müşteri


In [626]:
data_rfm[["Segment","Recency","Frequency", "Monetary"]].groupby("Segment").agg(["mean","median","count"])

Recency               Frequency               \
                                mean median count       mean median count   
Segment                                                                     
Dikkat edilsin            417.212560  417.0   207   2.454106    2.0   207   
Gözde Müşteri             371.119155  371.0   663  12.499246    8.0   663   
Kaybedilmesin             487.717949  472.5    78   9.038462    7.5    78   
Potansiyel Sadık Müşteri  382.773256  383.0   516   2.023256    2.0   516   
Riskli                    515.937807  494.0   611   3.070376    3.0   611   
Sadık Müşteri             400.275908  395.0   743   6.816958    5.0   743   
Umut Vaat eden            389.758621  389.0    87   1.000000    1.0    87   
Uykuda                    417.851312  417.0   343   1.201166    1.0   343   
Uzun Süredir Pasif        578.309055  577.0  1016   1.126969    1.0  1016   
Yeni Müşteri              372.580000  372.5    50   1.000000    1.0    50   

                             Monetary                  
                                 mean    median count  
Segment                                                
Dikkat edilsin            1058.788068   730.470   207  
Gözde Müşteri             6852.264167  2508.320   663  
Kaybedilmesin             4072.967051  2316.485    78  
Potansiyel Sadık Müşteri   729.164806   523.660   516  
Riskli                    1188.207056   760.190   611  
Sadık Müşteri             2743.484342  1818.710   743  
Umut Vaat eden             368.020690   293.740    87  
Uykuda                     442.266851   317.760   343  
Uzun Süredir Pasif         401.982150   250.155  1016  
Yeni Müşteri               386.199200   258.825    50

In [632]:
data_rfm[data_rfm["Segment"] == "Sadık Müşteri"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
customer_id,,,,,,,,
12359,425,6,2563.36,3,4,5,345,Sadık Müşteri
12402,392,5,1031.30,4,4,4,444,Sadık Müşteri
12409,420,4,12346.62,3,4,5,345,Sadık Müşteri
12417,410,11,3167.81,3,5,5,355,Sadık Müşteri
12422,394,14,4428.24,4,5,5,455,Sadık Müşteri


In [ ]:
# SEÇİLEN SEGMENTLERİN PROFİLLENMESİ